In [4]:
import pickle
import menpo.io as mio
import os
from sklearn import svm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize


with open('/Programing/GR/Code/Python/models/au_models.pkl', 'rb') as input:
    models = pickle.load(input)

with open('/Programing/GR/Code/Python/models/facs.pkl', 'rb') as input:
    facs = pickle.load(input)
    
print(type(models))
#print(len(models))
    

<class 'list'>


In [5]:
path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'


class ChangeVector:
    def __init__(self, facs = [], landmarkChange = [], emotion = 0):
        self.landmarkChange = landmarkChange
        self.facs = facs
        self.emotion = emotion
        
def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

svm_testing_data = []
path_to_svm_testing_database = "/Programing/GR/Code/CK+/test-aam-images/**/**/**/*"
testing_images = mio.import_images(path_to_svm_testing_database, verbose=True)
testing_images = testing_images.map(process)

count = 0;
while(count < len(testing_images)):
    file_path = str(testing_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            if(int(float(x)) not in data_facs and int(float(x)) != 0):
                data_facs.append(int(float(x)))
    #print(array)
    fi.close()
    
    landmarkChange = []
    landmark_neutral = testing_images[count].landmarks['PTS'].lms.points
    landmark_perk = testing_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
    svm_testing_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   
    count = count + 2

Found 46 assets, index the returned LazyList to import.


In [9]:
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_testing_data:
        if(au in data.facs):
            y_label.append(1)
        else:
            y_label.append(0)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmarkChange:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    print(models[facs.index(au)].score(x_training, y_label))

0.913043478261
0.869565217391
0.739130434783
0.95652173913
0.826086956522
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.913043478261
1.0
1.0
0.95652173913
1.0
1.0
1.0
1.0
0.95652173913
0.95652173913
1.0
0.913043478261
1.0
1.0
1.0
1.0
1.0
1.0
0.913043478261
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [10]:
print(x_training[0])

[7.8054411249516136, -0.71547500419853094, 7.8525024761978983, -0.6463011330943047, 7.6253081786376242, -0.65827245603927054, 7.6824029031348289, -0.64285143185077942, 7.3717605761647178, -0.41685887787173215, 7.0874954582861278, 0.14987740740923527, 6.4067780971288073, 0.16068612569601726, 6.20243080107997, 0.15393250476068943, 4.6388646644512335, 0.16402446325653841, 3.7114587186735264, -0.87401329773226166, 3.9244705724858306, -1.8340131847457144, 4.1737600749395938, -2.7005683975813781, 4.5507967450342051, -3.6038643868855615, 5.5137344127211065, -4.0067901984060939, 6.2947102767401191, -4.0707894841663688, 7.0551508031627037, -4.2564955985740198, 7.8111079345984251, -4.2848430714809638, 7.4361852136755076, -3.6980817725968222, 3.6876464582333952, -4.9805590968298645, 1.9039689363972343, -4.1199932418242469, -0.90987567025456428, -2.9332711032417791, -1.3112567766804304, -1.5953454159952685, 0.71566938079394049, -3.1698606408941714, 3.0568165367602091, -2.6490512315099011, 6.324576